# Instagram Graph API call

## 1. Get Credential (w/ dotenv)

To protect credential info, you have to make `.env` file  
Here's some examples  

1. INSTAGRAM_ACCESS_TOKEN
2. INSTAGRAM_BUSINESS_ACCOUNT
3. FB_APP_SECRET_CODE
4. FB_CLIENT_ID

In [2]:
import requests
import json
import os
from pprint import pprint as pp
from dotenv import load_dotenv

# load .env
load_dotenv()
IG_ACCESS_TOKEN=os.environ.get("IG_ACCESS_TOKEN")
FB_APP_SECRET_CODE=os.environ.get('FB_APP_SECRET_CODE')
IG_BUSINESS_ID=os.environ.get('IG_BUSINESS_ID')

def get_creds():
    creds = dict() 
    creds['access_token'] = IG_ACCESS_TOKEN   # access token for use with all api calls
    creds['client_secret'] = FB_APP_SECRET_CODE  # client secret from facebook app
    creds['graph_domain'] = 'https://graph.facebook.com/' # base domain for api calls
    creds['graph_version'] = 'v20.0' # version of api we are hitting
    creds['instagram_account_id'] = IG_BUSINESS_ID # user business account id
    # with domain and version
    creds['endpoint_base'] = creds['graph_domain'] + creds['graph_version'] + '/' # base endpoint
    
    return creds

## 2. Create Media Object Response & Publish Media

### (Optional) Create Carousel post

Steps required for making a carousel post are.  
1. Upload every images or video you want to make a carousel of one by one and store the container_id in a list.  
2. Make another container that contains the container_id list.  
3. Publish the container.  

In [3]:
def create_media_object(params):
    """Create media `Container` object
    
    API Endpoint:
    - For image
        https://graph.facebook.com/
        v20.0/{ig-user-id}/media?image_url={image-url}&caption={caption}&access_token={access_token}
    
    - For video
        https://graph.facebook.com/
        v20.0/{ig-user-id}/media?video_url={video-url}&caption={caption}&access_token={access_token}

    Returns:
        json object: data from the endpoint
    """

    url = params['endpoint_base'] + params['instagram_account_id'] + '/media'

    endpointParams = dict()
    if params['media_type'] == 'image':
        endpointParams['image_url'] = params['media_url']
    else:
        endpointParams['video_url'] = params['media_url']
        endpointParams['media_type'] = params['media_type']
    
    endpointParams['caption'] = params['caption']
    endpointParams['access_token'] = params['access_token']
    
    response = requests.post(url, data=endpointParams)

    return response.json()



def publish_media(media_object_id, params):
    """ Publish `media Container` object
    
    API Endpoint:
        https://graph.facebook.com/
        v20.0/{ig-user-id}/media_publish?creation_id={creation-id}&access_token={access_token}

    Returns:
        json object: data from the endpoint
    """

    url = params['endpoint_base'] + params['instagram_account_id'] + '/media_publish'

    endpointParams = dict()
    endpointParams['creation_id'] = media_object_id
    endpointParams['access_token'] = params['access_token']

    response = requests.post(url, data=endpointParams)
    
    print("\n---- IMAGE MEDIA OBJECT PUBLISHED ---- \n")
    print("\tID:") # label
    print("\t" + response.json()['id']) # id of the object

    return response.json()

#### Example Post

In [43]:
# main.py
params=get_creds()
params['media_type'] = 'image' # there are 2 types in input('image', 'video')

params['media_url'] = 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSCK8YiyxtZvfmGrgrxdwbLG1z7o9VxdI5Nkw&s' # sample image with high 
params['caption'] = 'This image was posted through the Instagram Graph API with a python script, test3 \n  #안정성 테스트 #해치웠나?'

image_media_object_response = create_media_object(params)
image_media_id = image_media_object_response['id']
published_media_id = publish_media(image_media_id, params)

## 3. Get User Media & Comments

In [36]:
def get_user_media(params):
    """ Get User Media id
    
    API Endpoint:
        https://graph.facebook.com/
        v20.0/{ig-user-id}?fields=media&access_token={access_token}
    """
    
    url = params['endpoint_base'] + params['instagram_account_id'] + '/media'
    
    endpointParams = dict()
    endpointParams['access_token'] = params['access_token']
    endpointParams['fields'] = 'media'
    response = requests.get(url, endpointParams)
    return response.json()



def get_media_comments(media_id, params):
    """  Get media comments

    API Endpoint:
        https://graph.facebook.com/v20.0/{media_id}/comments?access_token={access_token}
    """
    url = params['endpoint_base'] + media_id + '/comments?'
    endpointParams = dict()
    endpointParams['access_token'] = params['access_token']

    response = requests.get(url, endpointParams)
    return response.json()

### 

In [37]:
from pprint import pprint as pp

my_media = get_user_media(params)
comments_data = []

for data in my_media['data']:
    temp_media_id = data['id']
    comments_data.append(get_media_comments(temp_media_id, params))

pp(comments_data)

[{'data': [{'id': '18011557040594782',
            'text': '댓글을 작성해봅시다',
            'timestamp': '2024-07-17T19:17:41+0000'}]},
 {'data': []},
 {'data': []},
 {'data': []}]


In [44]:
# 댓글 작성 - 내일 참고하며 다시 해볼 것.
"""
https://medium.com/@maanideeprkummiitha/a-beginners-guide-to-interacting-with-instagram-using-python-and-the-graph-api-8fe7956fcc   # create api function tutorial

https://velog.io/@shi9476/Instagram-API-%EC%97%B0%EB%8F%99               # 참고자료 1 - long live token 자동재발급

https://developers.facebook.com/docs/instagram-platform/instagram-graph-api/content-publishing#----------            # publishing

https://developers.facebook.com/docs/instagram-platform/instagram-graph-api/reference/ig-user/media_publish         # replied comment

https://developers.facebook.com/docs/instagram-platform/instagram-graph-api/comment-moderation""" # 댓글에 답글 작성


'\nhttps://medium.com/@maanideeprkummiitha/a-beginners-guide-to-interacting-with-instagram-using-python-and-the-graph-api-8fe7956fcc   # create api function tutorial\n\nhttps://velog.io/@shi9476/Instagram-API-%EC%97%B0%EB%8F%99               # 참고자료 1 - long live token 자동재발급\n\nhttps://developers.facebook.com/docs/instagram-platform/instagram-graph-api/content-publishing#----------            # publishing\n\nhttps://developers.facebook.com/docs/instagram-platform/instagram-graph-api/reference/ig-user/media_publish         # replied comment\n\nhttps://developers.facebook.com/docs/instagram-platform/instagram-graph-api/comment-moderation'